In [1]:
# 本代码由可视化策略环境自动生成 2021年10月9日 16:16
# 本代码单元只能在可视化模式下编辑。您也可以拷贝代码，粘贴到新建的代码单元或者策略，然后修改。


# Python 代码入口函数，input_1/2/3 对应三个输入端，data_1/2/3 对应三个输出端
def m5_run_bigquant_run(input_1, start_date,end_date):
    # 示例代码如下。在这里编写您的代码
    ins=input_1.read_pickle()['instruments']
    start_date=input_1.read_pickle()['start_date']
    end_date=input_1.read_pickle()['end_date']
#    print(ins, start_date, end_date)
    df=DataSource('bar1d_CN_FUND').read(ins, start_date=start_date, end_date=end_date)
    #df = df[['date', 'close', 'instrument']].rename(columns={'close': 'close_0'})
    data_1 = DataSource.write_df(df)
    return Outputs(data_1=data_1, data_2=None, data_3=None)

# 后处理函数，可选。输入是主函数的输出，可以在这里对数据做处理，或者返回更友好的outputs数据格式。此函数输出不会被缓存。
def m5_post_run_bigquant_run(outputs):
    return outputs

# 回测引擎：初始化函数，只执行一次
def m3_initialize_bigquant_run(context):
    import numpy as np
    import pandas as pd
    from scipy.optimize import minimize

    pd.set_option('display.max_columns', None)  # 最多所有列
    pd.set_option('display.max_rows', None)  # 显示所有行
    # pd.set_option('display.float_format',lambda x: '%.6f'%x) # #显示小数位数两位
    pd.set_option('display.width', 1024)  # 显示宽度
    # pd.set_option('precision', 1)   # 设置显示数值的精度

    import warnings
    warnings.filterwarnings('ignore')  # 关闭运行时的警告
    np.set_printoptions(linewidth=100, suppress=True)   # 打印numpy时设置显示宽度，并且不用科学计数法显示
    print('载入库完成')
    # 加载仓位数据
    # context.ratios_dict = context.options['data'].read_pickle()
    
    # 系统已经设置了默认的交易手续费和滑点，要修改手续费可使用如下函数
    context.set_commission(PerOrder(buy_cost=0.00001, sell_cost=0.00001, min_cost=0.2))
    context.original_position = 0

def get_cov_mat(secIDs, data, end_date):
    """
    基于代码获取历史数据的年化协方差矩阵
    """
    df_rtn = pd.DataFrame()
    start_date = (pd.to_datetime(end_date) - datetime.timedelta(days=250)).strftime('%Y-%m-%d')  # 向前推150天
#     data['date_parsed'] = pd.to_datetime(data['date'],format="%Y-%m-%d")
#     df = data[(data['date_parsed'] >= start_date) & (data['date_parsed'] <= end_date)].copy()
#     del df['date_parsed']
#     print(df)
    for i in range(len(secIDs)):
        df_bar = DataSource('bar1d_CN_FUND').read(secIDs[i], start_date=start_date, end_date=end_date, fields = ['instrument', 'date', 'close'])  # 查询历史行情
#         df_bar = df[df['instrument'] == secIDs].copy()
        df_bar.dropna(inplace=True)
        if secIDs[i] == '512690.HOF':  # 解决行情的除权，采取后复权
                df_bar['close'] = df_bar.apply(lambda x: round(x['close'] / 1.961, 3) if x['date'] <= pd.to_datetime('2021-05-14') else x['close'], axis=1)  # 512690.HOF 2021-05-14 之前数据 / 1.961
        elif secIDs[i] == '512010.HOF':
            df_bar['close'] = df_bar.apply(lambda x: round(x['close'] / 3.905, 3) if x['date'] <= pd.to_datetime('2021-06-25') else x['close'], axis=1)  # 512010.HOF 2021-06-25 之前数据 / 3.905
        elif secIDs[i] == '159928.ZOF':
            df_bar['close'] = df_bar.apply(lambda x: round(x['close'] / 4, 3) if x['date'] <= pd.to_datetime('2021-06-24') else x['close'], axis=1)  # 159928.ZOF 2021-06-24 之前数据 / 4
#         print(len(df_bar))
#         print(df_bar)
        if len(df_bar) >= 100:  # 去掉历史记录太少的数据
            df_bar[secIDs[i]] = np.log(df_bar['close'] / df_bar['close'].shift(1))
#             df = df_bar[['date',secIDs[i]]].copy()
            if df_rtn.empty:
                df_rtn = df_bar[['date', secIDs[i]]].copy()
            else:
#                 print(df_rtn.head(3))
                df_rtn = df_rtn.merge(df_bar[['date',secIDs[i]]], on=['date'], how='outer')
            
    df_rtn.set_index('date', drop=True, inplace=True)
    df_rtn = df_rtn[1:]
    df_rtn.fillna(0, inplace=True)
    return df_rtn.ewm(20).cov()*252, df_rtn*252  # 一年大概252个交易日，采取年化是业内通用做法
    
def rebalance_signal(portfolio, original_position, today):
    """
    总体的仓位控制
    """
    if portfolio.cash / portfolio.portfolio_value > 0.25:
        print(today, '：现金仓位超过25%，触发重新配仓')
        return True
    elif (portfolio.positions_value - original_position)/original_position < -0.1:
        print(today, '：仓位跌幅超过10%，触发重新配仓')
        return True
    elif (portfolio.positions_value - original_position)/original_position > 0.25:
        print(today, '：仓位涨幅超过25%，触发重新配仓')
        return True
    else:
        return False

def get_before_tradeday(date, n):
    """
    获取指定日期前的交易日日期
    """
    today = date
    today_datetime = datetime.datetime.strptime(today,"%Y-%m-%d")
    past_datetime = today_datetime - datetime.timedelta(days=n)
    past = past_datetime.strftime("%Y-%m-%d")
    return past

def valudate_input_type(valudate_data, valudate_type):
    """
    格式校验
    """
    if not isinstance(valudate_data, valudate_type):
        print('输入矩阵不匹配校验格式', str(valudate_type))
        return False
    else:
        return True
    
def calculate_risk_contribution(w,V):
    """
    计算资产对组合风险的贡献
    """
    w = np.matrix(w)
    sigma = np.sqrt(calculate_portfolio_var(w,V))
    # Marginal Risk Contribution
    MRC = V*w.T
    # Risk Contribution
    RC = np.multiply(MRC,w.T)/sigma
    return RC
    
def get_weight_scipy_opt(cov_mat, exp_rtn, method, wts_adjusted):
    '''
    功能：输入协方差矩阵，得到不同优化方法下的权重配置
    输入：
        cov_mat  pd.DataFrame,协方差矩阵，index和column均为资产名称
        method  优化方法，可选的有 min_var、risk_parity、max_div、mean_weights、mean_vol
    输出：
        pd.Series  index为资产名，values为weight
    '''
#     if not valudate_input_type(cov_mat, pd.DataFrame):
#         return
    from scipy.optimize import minimize
    omega = np.matrix(cov_mat.values)  # 协方差矩阵
    
    def fun1(x):
        # 最小方差
#         return np.matrix(x) * omega * np.matrix(x).T
        return np.dot(x.T, np.dot(cov_mat,x))  # 这里的风险是一个对投资组合风险的总衡量
    
    def fun2(x):
        # 风险平价
        tmp = (omega * np.matrix(x).T).A1
#         risk = x * tmp
        risk = x * tmp/ np.sqrt(np.matrix(x) * omega * np.matrix(x).T).A1[0]
        delta_risk = [sum((i - risk)**2) for i in risk]
#         print(sum(delta_risk))
        return sum(delta_risk)
    
    def fun3(x):
        # 最大分散比
        x = np.asmatrix(x)
        cov = np.asmatrix(omega)
        vol = np.diagonal(cov)
        w_vol = np.sum(np.multiply(x, vol))
        port_vol = np.sqrt((x * cov * x.T)[0, 0])
        return -w_vol/port_vol
    
    # 初始值 + 约束条件 
    x0 = np.ones(omega.shape[0]) / omega.shape[0]  
    bnds = tuple((0, None) for x in x0)
    cons = ({'type':'eq', 'fun': lambda x: np.sum(x) - 1.0},)  # 总权重约束
    cons = cons + ({'type': 'ineq', 'fun': lambda x: x},)  # 仅做多约束
    cons = cons + ({'type': 'ineq', 'fun': lambda x: 0.2 - x},)  # 单仓位约束不高于20%
    options={'disp':False, 'maxiter':1000, 'ftol':1e-20}
    
    # 执行优化解算
    if method in['min_var', 'risk_parity', 'max_div']:
        if method == 'min_var':  # 组合方差最小时权重
#             print('try minimize...')
            res = minimize(fun1, x0, bounds=bnds, constraints=cons, method='SLSQP', options=options)
        elif method == 'risk_parity':  # 风险平价
            res = minimize(fun2, x0, bounds=bnds, constraints=cons, method='SLSQP', options=options)
        elif method == 'max_div':  # 最大分散比
            bnds = [(max(0, x[0]), x[1]) for x in bnds]
            res = minimize(fun3, x0, bounds=bnds, constraints=cons, method='SLSQP', options=options)
#         print(res)
        if not res['success']:
            print('有多解存在，返回次优解，调仓方案需人工斟酌')
        wts = pd.Series(index=cov_mat.index, data=np.array(np.round(res['x'],3)).flatten())  # 大部分情况下无最优解，只能给出次优解
    else:
        if method == 'mean_weights':  # 平均分配权重
            res = np.array([1.0 / exp_rtn.shape[1]] * exp_rtn.shape[1])
        elif method == 'mean_vol':  # 等波动率
            res = 1/np.sqrt(exp_rtn.std())
            res =  res/res.sum()
        wts = pd.Series(index=cov_mat.index, data= res)
    
    # 权重调整
    if wts_adjusted:
        wts = wts[wts >= 0.02]
        return wts / wts.sum() * 1.0
    elif not wts_adjusted:
        return wts
# 回测引擎：每日数据处理函数，每天执行一次
def m3_handle_data_bigquant_run(context, data):
    today = data.current_dt.strftime('%Y-%m-%d')  # 获取今日的日期字符串
    if rebalance_signal(context.portfolio, context.original_position, today) or context.trading_day_index % 21 == 0:  # 按月调仓
#     if rebalance_signal(context.portfolio, context.original_position) or data.current_dt.weekday() == 0:  # 按周调仓
        print('=' * 100)
        cash = context.portfolio.cash  # 现金
        cur_position = {e.symbol: p.available_close_amount
                 for e, p in context.portfolio.positions.items()}
        
        # 核心优化逻辑
        cov, rtn = get_cov_mat(context.instruments, context.data, today)
        cov = cov.sort_index()
        cov = cov.loc[today, :]  # 基于加权协方差矩阵，取当天的结果
        cov.index = cov.index.droplevel()
        try:
#             print(rtn)
#             print(cov)
            # min_var、risk_parity、mean_weights、mean_vol、max_div
            wts = get_weight_scipy_opt(cov, rtn, method='min_var', wts_adjusted=True)
        except Exception as e:
            print(e)
            return
        
        if len(wts):
            print('日期：', today, '，当前现金：', cash, '，仓位：', cur_position)
            print('交易计划：', wts)
            
            # 交易逻辑
            stock_hold_now = [equity.symbol for equity in context.portfolio.positions]  # 获取目前持仓的股票列表
            sell_list = [stk for stk in stock_hold_now if stk not in list(wts.keys())]  # 确认是否有不在买入名单中的票，有就全部卖掉
            
            for stk in sell_list:
                #             print('卖出股票：', sell_list)
                context.order_target(stk, 0)

            for key, value in wts.items():  # 按照权重方案，完成买入
                #             print('买入股票和仓位：', key, value)
                try:
                    if data.can_trade(context.symbol(key)):
                        context.order_target_percent(key, value)
                        context.original_position = context.portfolio.positions_value  # 保存当前调仓后的市值
                except:
                    print(key, '买入失败')
        
    else:
        return
# 回测引擎：准备数据，只执行一次
def m3_prepare_bigquant_run(context):
    # 加载预测数据
    df = context.options['data'].read_df()
    context.data = df
    pass

# 回测引擎：每个单位时间开始前调用一次，即每日开盘前调用一次。
def m3_before_trading_start_bigquant_run(context, data):
    
    pass

m1 = M.input_features.v1(
    features="""# #号开始的表示注释
# 多个特征，每行一个，可以包含基础特征和衍生特征
#buy_condition=where(mean(close_0,5)>mean(close_0,10),1,0)
#sell_condition=where(mean(close_0,5)<mean(close_0,10),1,0)
close
#open
#daily_return_0""",
    m_cached=False
)

m2 = M.instruments.v2(
    start_date=T.live_run_param('trading_date', '2021-01-01'),
    end_date=T.live_run_param('trading_date', '2021-10-01'),
    market='CN_FUND',
    instrument_list="""159995.ZOF
159928.ZOF
512700.HOF
512880.HOF
512690.HOF
512010.HOF
512680.HOF
512980.HOF
515700.HOF
512400.HOF
512580.HOF
515050.HOF
159825.ZOF
588000.HOF
159870.ZOF
513100.HOF
513050.HOF
159819.ZOF
512200.HOF
159996.ZOF
159920.ZOF
162411.ZOF""",
    max_count=0
)

m5 = M.cached.v3(
    input_1=m2.data,
    run=m5_run_bigquant_run,
    post_run=m5_post_run_bigquant_run,
    input_ports='',
    params='{}',
    output_ports=''
)

m8 = M.derived_feature_extractor.v3(
    input_data=m5.data_1,
    features=m1.data,
    date_col='date',
    instrument_col='instrument',
    drop_na=True,
    remove_extra_columns=True
)

m3 = M.trade.v4(
    instruments=m2.data,
    options_data=m8.data,
    start_date='',
    end_date='',
    initialize=m3_initialize_bigquant_run,
    handle_data=m3_handle_data_bigquant_run,
    prepare=m3_prepare_bigquant_run,
    before_trading_start=m3_before_trading_start_bigquant_run,
    volume_limit=0.025,
    order_price_field_buy='open',
    order_price_field_sell='open',
    capital_base=1000000,
    auto_cancel_non_tradable_orders=True,
    data_frequency='daily',
    price_type='后复权',
    product_type='股票',
    plot_charts=True,
    backtest_only=False,
    benchmark='000300.INDX'
)


[2021-10-09 15:47:40.285301] INFO: moduleinvoker: input_features.v1 开始运行..

[2021-10-09 15:47:40.329018] INFO: moduleinvoker: input_features.v1 运行完成[0.043716s].

[2021-10-09 15:47:40.337180] INFO: moduleinvoker: instruments.v2 开始运行..

[2021-10-09 15:47:40.363639] INFO: moduleinvoker: instruments.v2 运行完成[0.026444s].

[2021-10-09 15:47:40.382169] INFO: moduleinvoker: cached.v3 开始运行..

...

[2021-10-09 15:49:30.409061] INFO: Performance: Simulated 182 trading days out of 182.

[2021-10-09 15:49:30.413224] INFO: Performance: first open: 2021-01-04 09:30:00+00:00

[2021-10-09 15:49:30.415816] INFO: Performance: last close: 2021-09-30 15:00:00+00:00

[2021-10-09 15:49:31.550752] INFO: moduleinvoker: backtest.v8 运行完成[106.787689s].

[2021-10-09 15:49:31.553149] INFO: moduleinvoker: trade.v4 运行完成[109.712525s].